In [1]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

model = ChatGoogleGenerativeAI(model='gemini-2.0-flash-lite', google_api_key=os.environ["GOOGLE_API_KEY"])

In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path=("https://en.wikisource.org/wiki/Analysis_and_Assessment_of_Gateway_Process")
)

docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://en.wikisource.org/wiki/Analysis_and_Assessment_of_Gateway_Process', 'title': 'Analysis and Assessment of Gateway Process - Wikisource, the free online library', 'language': 'en'}, page_content='\n\n\n\nAnalysis and Assessment of Gateway Process - Wikisource, the free online library\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain PageCommunity portalCentral discussionRecent changesSubject indexAuthorsRandom workRandom authorRandom transcriptionHelpSpecial pages\n\n\n\n\n\n\t\tDisplay Options\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\

In [3]:
text = docs[0].page_content
len(text)

94523

In [4]:
count = 0
divided = "<sep>"
for i in range(len(text)):
    if count == 2000:
        divided += "<sep>"
        count = 0
    else:
        count += 1
        divided += text[i]

In [5]:
divided

'<sep>\n\n\n\nAnalysis and Assessment of Gateway Process - Wikisource, the free online library\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain PageCommunity portalCentral discussionRecent changesSubject indexAuthorsRandom workRandom authorRandom transcriptionHelpSpecial pages\n\n\n\n\n\n\t\tDisplay Options\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAnalysis and Assessment of Gateway Process\n\n\n\nAdd languages\n\n\n\n\n\nAdd links\n\n\n\n\n\n\n\n\n\n\n\nPageSourceDiscussion\n\n\n\n\n\nEnglish\n\n\n

In [6]:
len(divided.split("<sep>"))

49

In [7]:
len(text)/2000

47.2615

In [8]:
# mmm close enough. perfect

In [ ]:
prompt = f"""You are a preprocessing assistant. Your task is to rewrite text chunks into self-contained units that can be understood without additional context. Do not add new information; only restate or clarify using the text itself."

The following text contains multiple chunks separated by the token <sep>.  

Task:  
- For each chunk, rewrite it so it is fully self-contained and understandable without other chunks.  
- Do not add new facts, only restate missing references.  
- Keep meaning and details intact.  
- Output each rewritten chunk in order, labeled clearly.  
- KEEP THE NUMBER OF CHUNKS SAME AND INTACT.

Text: {divided}
Example output(JSON):
{{
    "0":"Chunk1",
    "1":"Chunk2",
}}
"""

contextualized2 = model.invoke(prompt)


In [34]:
print(f"{contextualized2.content}")

```json
{
  "0": "Analysis and Assessment of Gateway Process - Wikisource, the free online library.\n\nThis document is a report titled \"Analysis and Assessment of Gateway Process,\" published in June 1983 by the US Army Operational Group, US Army Intelligence and Security Command, Fort George G. Meade, Maryland. It was written by Wayne M. McDonnell, LTC, MI, Commander, Det O. The report was written in response to a request for an assessment of the Gateway Experience, focusing on its mechanics and practicality. The author explains the need for extensive research and analysis to understand how and why the process works, including the use of biomedical models, quantum mechanics, and the science of out-of-body states. The report aims to provide a scientifically valid model of the time-space dimension and how expanded human consciousness transcends it to achieve Gateway's objectives. The report also addresses the impact of the Gateway Experience on common belief systems, aiming to show th

In [31]:
len(contextualized2.content)

43314

In [ ]:
(90000-43000)/90000
# a 50% reduction in the fucking text size.

0.5222222222222223

In [115]:
data = contextualized2.content

NameError: name 'contextualized2' is not defined

In [36]:
data += "}"

In [ ]:
# Now to vectorize, and store it in DB. much better results on this text as much as I have seen till now. 
# first we extract the whole shit. Using what?
json_data = "{"+ data[12:]

In [62]:
json_data = json_data[:-1] + '"}'
json_data

'{"0": "Analysis and Assessment of Gateway Process - Wikisource, the free online library.\\n\\nThis document is a report titled \\"Analysis and Assessment of Gateway Process,\\" published in June 1983 by the US Army Operational Group, US Army Intelligence and Security Command, Fort George G. Meade, Maryland. It was written by Wayne M. McDonnell, LTC, MI, Commander, Det O. The report was written in response to a request for an assessment of the Gateway Experience, focusing on its mechanics and practicality. The author explains the need for extensive research and analysis to understand how and why the process works, including the use of biomedical models, quantum mechanics, and the science of out-of-body states. The report aims to provide a scientifically valid model of the time-space dimension and how expanded human consciousness transcends it to achieve Gateway\'s objectives. The report also addresses the impact of the Gateway Experience on common belief systems, aiming to show that it

In [63]:
# This approach seems to be shit as the llm can produce anything, and again we have no output checkers. I may need structured output.

In [ ]:
# STRUCTURED OUTPUT?
# Man. 

In [1]:
# Trying again. But with giving the number of chunks along with it. 

In [10]:
prompt = f"""You are a preprocessing assistant. Your task is to rewrite text chunks into self-contained units that can be understood without additional context. Do not add new information; only restate or clarify using the text itself."

The following text contains multiple chunks separated by the token <sep>.  

Task:  
- For each chunk, rewrite it so it is fully self-contained and understandable without other chunks.  
- Do not add new facts, only restate missing references.  
- Keep meaning and details intact.  
- Output each rewritten chunk in order, labeled clearly.  
- KEEP THE NUMBER OF CHUNKS SAME AND INTACT.

Text: {divided}
Number of Chunks: {len(divided.split("<sep>"))}
Example output(JSON):
{{
    "0":"Chunk1",
    "1":"Chunk2",
}}
"""

contextualized3 = model.invoke(prompt)


In [11]:
print(f"{contextualized3}")

content='```json\n{\n  "0": "Analysis and Assessment of Gateway Process - Wikisource, the free online library. This text is a document from Wikisource, a free online library, and details an analysis and assessment of the Gateway Process.",\n  "1": "The author explains that to assess the Gateway Experience, they needed to research and analyze how and why the process works, which proved to be difficult. They used biomedical models from Itzhak Bentov and delved into quantum mechanics to describe human consciousness, time-space dimensions, and out-of-body states.",\n  "2": "The author begins by profiling the fundamental biomedical factors affecting techniques like hypnosis, biofeedback, and transcendental meditation to compare them with the Gateway experience. This introductory material supports the paper\'s conclusions and indicates that these related techniques may provide entry points to accelerate movement into the Gateway Experience.",\n  "3": "The author notes that understanding the 

In [11]:
# PERFECT!!!!!!

In [12]:
len(contextualized3.content)

22108

In [13]:
(90000-22000)/90000

0.7555555555555555

In [14]:
# 40% decrease in text length. mmmm. Sounds Not, Good. Cuz every word in the actual document is "RELEVANT"
# Now on to this. Does metadata also gets vectorized? Should I inject it into metadata OR append to the actual document content. 

In [ ]:
# Ight bet. We add it to content, cuz the metadata is NOT vectorized.
# Now.


In [14]:
json_string = "{" + contextualized3.content[12:]
json_string 

'{"0": "Analysis and Assessment of Gateway Process - Wikisource, the free online library. This text is a document from Wikisource, a free online library, and details an analysis and assessment of the Gateway Process.",\n  "1": "The author explains that to assess the Gateway Experience, they needed to research and analyze how and why the process works, which proved to be difficult. They used biomedical models from Itzhak Bentov and delved into quantum mechanics to describe human consciousness, time-space dimensions, and out-of-body states.",\n  "2": "The author begins by profiling the fundamental biomedical factors affecting techniques like hypnosis, biofeedback, and transcendental meditation to compare them with the Gateway experience. This introductory material supports the paper\'s conclusions and indicates that these related techniques may provide entry points to accelerate movement into the Gateway Experience.",\n  "3": "The author notes that understanding the physics of altered hu

In [ ]:
json_string2 = json_string[:-7] + '"}'
json_string2

'{"0": "Analysis and Assessment of Gateway Process - Wikisource, the free online library. This text is a document from Wikisource, a free online library, and details an analysis and assessment of the Gateway Process.",\n  "1": "The author explains that to assess the Gateway Experience, they needed to research and analyze how and why the process works, which proved to be difficult. They used biomedical models from Itzhak Bentov and delved into quantum mechanics to describe human consciousness, time-space dimensions, and out-of-body states.",\n  "2": "The author begins by profiling the fundamental biomedical factors affecting techniques like hypnosis, biofeedback, and transcendental meditation to compare them with the Gateway experience. This introductory material supports the paper\'s conclusions and indicates that these related techniques may provide entry points to accelerate movement into the Gateway Experience.",\n  "3": "The author notes that understanding the physics of altered hu

In [16]:
print(type(json_string))   

<class 'str'>


In [17]:
import json
data = json.loads(json_string2)

In [18]:
data

{'0': 'Analysis and Assessment of Gateway Process - Wikisource, the free online library. This text is a document from Wikisource, a free online library, and details an analysis and assessment of the Gateway Process.',
 '1': 'The author explains that to assess the Gateway Experience, they needed to research and analyze how and why the process works, which proved to be difficult. They used biomedical models from Itzhak Bentov and delved into quantum mechanics to describe human consciousness, time-space dimensions, and out-of-body states.',
 '2': "The author begins by profiling the fundamental biomedical factors affecting techniques like hypnosis, biofeedback, and transcendental meditation to compare them with the Gateway experience. This introductory material supports the paper's conclusions and indicates that these related techniques may provide entry points to accelerate movement into the Gateway Experience.",
 '3': "The author notes that understanding the physics of altered human cons

In [19]:
type(data)

dict

In [20]:
len(data)

50

In [33]:
for i in range(len(data)):
    print(data[str(i)])

Analysis and Assessment of Gateway Process - Wikisource, the free online library. This is the main title of the document.
The author states that in order to assess the validity and practicality of the Gateway Experience, they needed to research and analyze the process to fully understand its mechanics. This proved to be a difficult task. The author used biomedical models from Itzhak Bentov to understand the physical aspects, delved into quantum mechanics to describe human consciousness, and used physics to explain out-of-body states. The goal was to remove the stigma of occult connotations and provide an objective assessment.
The author began by profiling fundamental biomedical factors related to techniques like hypnosis, biofeedback, and transcendental meditation to compare their objectives and functioning with the Gateway Experience. This introductory material supports the paper's conclusions and suggests that these techniques may accelerate entry into the Gateway Experience.
The aut

In [34]:
LETS FUCKIN GO. LETS. FUCKINg GO.

SyntaxError: invalid syntax (2086458351.py, line 1)

In [35]:
LETS.

SyntaxError: invalid syntax (4165471179.py, line 1)

In [36]:
FUCKING

NameError: name 'FUCKING' is not defined

In [37]:
GO.

SyntaxError: invalid syntax (1367301457.py, line 1)

In [21]:
original_data = divided.split("<sep>")
original_data 

['',
 '\n\n\n\nAnalysis and Assessment of Gateway Process - Wikisource, the free online library\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain PageCommunity portalCentral discussionRecent changesSubject indexAuthorsRandom workRandom authorRandom transcriptionHelpSpecial pages\n\n\n\n\n\n\t\tDisplay Options\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAnalysis and Assessment of Gateway Process\n\n\n\nAdd languages\n\n\n\n\n\nAdd links\n\n\n\n\n\n\n\n\n\n\n\nPageSourceDiscussion\n\n\n\n\n\nEnglish\n\n\

In [ ]:
# TO make sure that there is no EMPTY string / chunk at the first, lets not put <sep> at the start.

In [23]:
len(original_data)

49

In [ ]:
# and because the first string was empty, it produced one more in the output. So lets remove the first one from generated text. Or maybe Not. Hmmm. It gives a good introduction. Mmm.

In [22]:
meta = list(data.values())
meta

['Analysis and Assessment of Gateway Process - Wikisource, the free online library. This text is a document from Wikisource, a free online library, and details an analysis and assessment of the Gateway Process.',
 'The author explains that to assess the Gateway Experience, they needed to research and analyze how and why the process works, which proved to be difficult. They used biomedical models from Itzhak Bentov and delved into quantum mechanics to describe human consciousness, time-space dimensions, and out-of-body states.',
 "The author begins by profiling the fundamental biomedical factors affecting techniques like hypnosis, biofeedback, and transcendental meditation to compare them with the Gateway experience. This introductory material supports the paper's conclusions and indicates that these related techniques may provide entry points to accelerate movement into the Gateway Experience.",
 "The author notes that understanding the physics of altered human consciousness requires n

In [24]:
len(meta)

50

In [ ]:
result = []
max_len = max(len(original_data), len(meta))
for i in range(max_len):
    x = original_data[i] if i < len(original_data) else ""
    y = meta[i] if i < len(meta) else ""
    result.append(f"{x}. Document Meaning : {y}")


In [26]:
result

['. Document Meaning : Analysis and Assessment of Gateway Process - Wikisource, the free online library. This text is a document from Wikisource, a free online library, and details an analysis and assessment of the Gateway Process.',
 '\n\n\n\nAnalysis and Assessment of Gateway Process - Wikisource, the free online library\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain PageCommunity portalCentral discussionRecent changesSubject indexAuthorsRandom workRandom authorRandom transcriptionHelpSpecial pages\n\n\n\n\n\n\t\tDisplay Options\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n

In [27]:
print(f"{result[2]}")

rting research and analysis to fully understand how and why the process works. Frankly, sir, that proved to be an extremely involved and difficult business. Initially, based on conversations with a physician who took the Gateway training with me, I had recourse to the biomedical models developed by Itzhak Bentov to obtain information concerning the physical aspects of the process. Then I found it necessary to delve into various sources for information concerning quantum mechanics in order to be able to describe the nature and functioning of human consciousness. I had to be able to construct a scientifically valid and reasonably lucid model of the time-space dimension and the means by which expanded human consciousness transcends it in achieving Gateway's objectives. Finally, I again found it necessary to use physics to bring the whole phenomenon of out-of-body states into the language of physical science to remove the stigma of its occult connotations, and put it in a frame of referenc

In [54]:
#Perfect. And also i am realising that it doesnt really matter THAT much how the content is mapped. cuz at the end, we ARE getting all the documents arent we. 
# TIme for documenting all these documents lol

In [28]:
from langchain_core.documents import Document

docs = [Document(page_content=item) for item in result]

In [29]:
docs

[Document(metadata={}, page_content='. Document Meaning : Analysis and Assessment of Gateway Process - Wikisource, the free online library. This text is a document from Wikisource, a free online library, and details an analysis and assessment of the Gateway Process.'),
 Document(metadata={}, page_content='\n\n\n\nAnalysis and Assessment of Gateway Process - Wikisource, the free online library\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain PageCommunity portalCentral discussionRecent changesSubject indexAuthorsRandom workRandom authorRandom transcriptionHelpSpecial pages\n\n\n\n\n\n\t\tDisplay Options\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create ac

# LETS START THE EMBEEDDING. 

In [30]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [31]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
vector_store = FAISS.from_documents(docs, embeddings)

In [32]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':4})

In [33]:
retriever.invoke("Frequency following response")

[Document(id='1c47accd-9b73-4bc1-8d8f-6749f542ad67', metadata={}, page_content='ation thus received through the same fundamental matrix by which it makes sense of ordinary physical sensory input to achieve meaning in a cognitive context. Such meaning is usually perceived visually in the form of symbols but may also be perceived as astonishing flashes of holistic intuition or even in the form \u200bof scenarios involving both visual and aural perception. The mechanics by which the mind exercises the consciousness function will be addressed in more detail later in this paper.\n7. Frequency Following Response. To achieve synchronization of brain hemispheres, the Hemi-Sync technique takes advantage of a phenomenon known as the Frequency Following Response(FFR) which means that if a subject hears a sound produced at a frequency which emulates one of those associated with the operation of the human brain, the brain will try to mimmick the same frequency pattern by adjusting its brainwave out

In [34]:
retriever.invoke("Focus 15")

[Document(id='e6d936a4-b742-41bf-a10b-2ffc6bfea9b1', metadata={}, page_content='ed in the foregoing study involves the following steps:\nA. Begin by using the Gateway Hemi-Sync tapes to achieve enhanced brain focus and to induce hemisphere synchronization.\nB. Then add strong REM sleep frequencies to induce left brain quiescence and deep physical relaxation.\nC. Provide hypnotic suggestion designed to enable an individual to induce deep autohypnotic state at will.\nD. Use autohypnotic suggestion to attain much enhanced focus of concentration and motivation in rapidly progressing through Focus 12 exercises.\nE. Then repeat steps A and B following use of the autohypnotic suggestion that an out-of-body movement will occur and be remembered.\nF. Repeat step E to achieve facility in gaining out-of-body state under conscious control. Alter hypnotic suggestion to stress ability to consciously control out-of-body movement and maintain it even after REM sleep state ends.\nG. Approach Focus 15 a

In [41]:
retriever.invoke("structure of universe")

[Document(id='4ac737a5-2a8d-4c95-9612-dc517ec8feaf', metadata={}, page_content='nsequences of the past and present can be seen coming together in the hologram such that the future can be predicted or "seen" with total accuracy). Moreover, it is possible to see how the implosion of energy patterns would cross and recross to create an incredibly complex four dimensional hologram or Torus, in spiral shape in reflection of the multi-dimensional developing pattern of evolution. All of the movements of the energies which comprise the universe leave their mark and hence tell their story throughout time.\n26. Quality of Consciousness. We noted earlier that the out-of-body state involves projection of a major portion of the energy pattern that represents human consciousness so that it may move either freely throughout the terrestrial sphere for purposes of information aquisition or into other dimensions outside of time-space, perhaps to interact with other forms of consciousness within the univ

In [35]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(template="""
You are a helpful assistant.
Answer ONLY from the provided transcript context.
If the context is insufficient, just say you don't know.
Context: {context}
Question: {question}
                        """, input_variables=['context', 'question'])

In [36]:
question = "Is structure of universe also explained in this? if yes, then explain it."
retrieved_docs = retriever.invoke(question)
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [37]:
context_text

'nsequences of the past and present can be seen coming together in the hologram such that the future can be predicted or "seen" with total accuracy). Moreover, it is possible to see how the implosion of energy patterns would cross and recross to create an incredibly complex four dimensional hologram or Torus, in spiral shape in reflection of the multi-dimensional developing pattern of evolution. All of the movements of the energies which comprise the universe leave their mark and hence tell their story throughout time.\n26. Quality of Consciousness. We noted earlier that the out-of-body state involves projection of a major portion of the energy pattern that represents human consciousness so that it may move either freely throughout the terrestrial sphere for purposes of information aquisition or into other dimensions outside of time-space, perhaps to interact with other forms of consciousness within the universe. Consciousness is the organizing and sustaining principle that provides th

In [38]:
# This is the final prompt
final_prompt = prompt.invoke({'context':context_text, "question":question})
final_prompt

StringPromptValue(text='\nYou are a helpful assistant.\nAnswer ONLY from the provided transcript context.\nIf the context is insufficient, just say you don\'t know.\nContext: nsequences of the past and present can be seen coming together in the hologram such that the future can be predicted or "seen" with total accuracy). Moreover, it is possible to see how the implosion of energy patterns would cross and recross to create an incredibly complex four dimensional hologram or Torus, in spiral shape in reflection of the multi-dimensional developing pattern of evolution. All of the movements of the energies which comprise the universe leave their mark and hence tell their story throughout time.\n26. Quality of Consciousness. We noted earlier that the out-of-body state involves projection of a major portion of the energy pattern that represents human consciousness so that it may move either freely throughout the terrestrial sphere for purposes of information aquisition or into other dimensio

In [39]:
# Model
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", google_api_key=os.environ["GOOGLE_API_KEY"])

In [40]:
answer = model.invoke(final_prompt)
answer.content

'Yes, the structure of the universe is explained. The universe is often called a "Torus" because it is thought to have the overall shape of an immense, self-contained spiral. It is a hologram where the implosion of energy patterns would cross and recross to create an incredibly complex four dimensional hologram or Torus, in spiral shape in reflection of the multi-dimensional developing pattern of evolution. The movements of the energies which comprise the universe leave their mark and hence tell their story throughout time.'

# Chain without memory

In [42]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [43]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [44]:

prompt = PromptTemplate(template="""
You are a helpful assistant.
Answer ONLY from the provided transcript context.
If the context is insufficient, just say you don't know.
Context: {context}
Question: {question}
                        """, input_variables=['context', 'question'])

parallel_chain = RunnableParallel(
    {
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
    }
)

parallel_chain.invoke("Topic")

{'context': 'ed in the foregoing study involves the following steps:\nA. Begin by using the Gateway Hemi-Sync tapes to achieve enhanced brain focus and to induce hemisphere synchronization.\nB. Then add strong REM sleep frequencies to induce left brain quiescence and deep physical relaxation.\nC. Provide hypnotic suggestion designed to enable an individual to induce deep autohypnotic state at will.\nD. Use autohypnotic suggestion to attain much enhanced focus of concentration and motivation in rapidly progressing through Focus 12 exercises.\nE. Then repeat steps A and B following use of the autohypnotic suggestion that an out-of-body movement will occur and be remembered.\nF. Repeat step E to achieve facility in gaining out-of-body state under conscious control. Alter hypnotic suggestion to stress ability to consciously control out-of-body movement and maintain it even after REM sleep state ends.\nG. Approach Focus 15 and 21 objectives(escape from time-space and interact \u200bwithin n

In [ ]:
parser = StrOutputParser()
normal_chain = parallel_chain | prompt | model | parser

In [47]:
main_chain = parallel_chain | normal_chain

In [48]:
normal_chain.invoke("How is the structure of universe explained in this document?")

'The universe is explained as a self-contained spiral, often called a "Torus," that evolved from the "Big Bang." Matter enters the ovoid pattern following ejection from a nucleus composed of extremely compressed energy through a "white hole" and departs via a "black hole." Time is a measure of the change that occurs as energy evolves into new, more complex forms as it progresses along the distance from the white hole side of the nucleus, around the shell of this "cosmic egg" until it enters the black hole.'

In [53]:
normal_chain.invoke("What was my last question?")

"I don't know."

# Adding History

In [106]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate


In [107]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [108]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [109]:
history_aware_retriever=create_history_aware_retriever(model,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000271AF319A90>, search_kwargs={'k': 4}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChun

In [ ]:
question_answer_chain=create_stuff_documents_chain(model,prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"is structure of universe covered? Explain it in detail"})
response

{'input': 'is structure of universe covered? Explain it in detail',
 'context': [Document(id='4ac737a5-2a8d-4c95-9612-dc517ec8feaf', metadata={}, page_content='nsequences of the past and present can be seen coming together in the hologram such that the future can be predicted or "seen" with total accuracy). Moreover, it is possible to see how the implosion of energy patterns would cross and recross to create an incredibly complex four dimensional hologram or Torus, in spiral shape in reflection of the multi-dimensional developing pattern of evolution. All of the movements of the energies which comprise the universe leave their mark and hence tell their story throughout time.\n26. Quality of Consciousness. We noted earlier that the out-of-body state involves projection of a major portion of the energy pattern that represents human consciousness so that it may move either freely throughout the terrestrial sphere for purposes of information aquisition or into other dimensions outside of t

In [112]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [113]:
conversational_rag_chain.invoke(
    {"input": "is structure of universe covered? Explain it in detail"},
    config={
        "configurable": {"session_id": "1"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Yes, the structure of the universe is covered in the provided text. Here\'s a detailed explanation:\n\n*   **Torus Shape:** The universe is often called a "Torus" because it is thought to have the overall shape of an immense, self-contained spiral.\n*   **Big Bang and Cosmic Egg:** The text uses the "Big Bang" theory as a starting point for describing the universe\'s evolution. It describes a process where matter is ejected from a nucleus through a "white hole," travels through the universe, and eventually enters a "black hole." This forms an ovoid or egg shape, often referred to as the "cosmic egg."\n*   **Hologram:** The text describes the universe as a hologram, where past and present events come together, allowing for future predictions.\n*   **Dimensions:** Between the Absolute and the "material" universe are various intervening dimensions to which human consciousness in altered states of being may gain access.\n*   **Energy:** The text explains that the universe is made of oscil

In [114]:
conversational_rag_chain.invoke(
    {"input": "Ohkk, so provide a summary about it"},
    config={
        "configurable": {"session_id": "1"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'The text discusses the concept of a universal hologram, often referred to as a "Torus," which is thought to have the overall shape of an immense, self-contained spiral. This model depicts the process of time-space evolution, drawing from the "Big Bang" theory. It also touches on the idea of altered states of consciousness and the ability to access different dimensions. The Gateway program is mentioned, which uses techniques to manipulate high-energy states and explore the past and future through different focus levels.'